In [1]:
# installing libraries
! pip install langchain-groq langchain sentence-transformers langchain-community pypdf chroma chromadb


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q langchain-groq faiss-gpu


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
import os
from sentence_transformers import SentenceTransformer
from google.colab import userdata
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever
from langchain.prompts import PromptTemplate


In [5]:
# Load the Constitution PDF
pdf_path = "/content/drive/MyDrive/The_Constitution_of_Kenya_2010.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [6]:
## initializing texts into chunks
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=50,
)
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

In [7]:
# initializing the embedding model
BGE_model = SentenceTransformer('BAAI/bge-small-en-v1.5')
class BGE_embeddings:
  def embed_documents(self, documents):
    return BGE_model.encode(documents, batch_size=8, normalized_embedding=True).tolist()
  def embed_query(self, query):
    return BGE_model.encode([query], normalized_embedding=True).tolist()[0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
## creating the vectorstores
constitution_vector_store = Chroma(
    collection_name = "constitution",
    embedding_function = BGE_embeddings(),
    persist_directory ="./chroma"
)

/tmp/ipython-input-2514688086.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  constitution_vector_store = Chroma(


In [9]:
## Creating in memory store
store = InMemoryStore()

In [10]:
## creating retrievers
retriever = ParentDocumentRetriever(
    vectorstore=constitution_vector_store,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [11]:
# Adding documents into the vectorstores
retriever.add_documents(documents)

In [12]:
# creating a child retreiver
child_retriever = constitution_vector_store.as_retriever()

In [13]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    groq_api_key=userdata.get('alice_in_wonderland')
)

In [14]:
#creating the template
template="""
 Answer the following questions according to the context given.
 If the question is out of context,just say the question is out of context Kindly give me another question.
 Do not try to answer a question out of context.
 {context}
 Question: {question}
"""

In [15]:
#creating the chain
qa_chain=PromptTemplate(template=template,input_variables=["context","question"])

In [16]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": qa_chain},
    return_source_documents=True
)

In [17]:
#  Test it
query = "What does the Constitution say about the right to life?"
answer=chain({"query":query})
print("Q:", query)
print("A:", answer)

/tmp/ipython-input-2486236122.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer=chain({"query":query})


Q: What does the Constitution say about the right to life?
A: {'query': 'What does the Constitution say about the right to life?', 'result': 'According to the Constitution of Kenya, 2010, the right to life is stated in Article 26 as follows:\n\n(1) Every person has the right to life.\n(2) The life of a person begins at conception.\n(3) A person shall not be deprived of life intentionally, except to the extent authorised by this Constitution or other written law.\n(4) Abortion is not permitted unless, in the opinion of a trained health professional, there is need for emergency treatment, or the life or health of the mother is in danger, or if permitted by any other written law.\n\nThis indicates that the Constitution recognizes the right to life as a fundamental right, and it begins at conception. However, it also allows for the deprivation of life in certain circumstances, such as when authorized by the Constitution or other written law.', 'source_documents': [Document(metadata={'produ

In [18]:
!pip install gradio

In [27]:
import gradio as gr

# keep a global chat history (list of tuples)
chat_history = []

def chat_fn(message, history):
    global chat_history

    # Run query on the RetrievalQA chain, with history
    response = chain.invoke({
        "query": message,
        "chat_history": chat_history
    })

    answer = response["result"]

    # Update chat history for follow-up context
    chat_history.append((message, answer))

    # Update Gradio chat UI
    history = history or []
    history.append((message, answer))
    return history, history

# Function to reset history
def clear_fn():
    global chat_history
    chat_history = []
    return None

with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="indigo")) as demo:
    with gr.Row():
        gr.Markdown(
            """
            <div style="text-align:center; padding:10px;">
                <h1 style="color:#1e3a8a;">🇰🇪 Kenyan Constitution Chatbot</h1>
                <p style="font-size:16px; color:gray;">Ask me anything about the Constitution of Kenya (2010).
                I can also handle follow-up questions!</p>
            </div>
            """
        )

    with gr.Row():
        with gr.Column(scale=4):
            chatbot = gr.Chatbot(
                height=450,
                show_label=False,
                bubble_full_width=False,
                avatar_images=("👤", "🤖")
            )
            msg = gr.Textbox(
                placeholder="Type your question here and press Enter...",
                show_label=False,
                lines=2
            )
            with gr.Row():
                clear = gr.Button("🧹 Clear Chat", variant="secondary")
                send = gr.Button("🚀 Send", variant="primary")

    # User sends a message (via Enter or Send button)
    msg.submit(chat_fn, [msg, chatbot], [chatbot, chatbot])
    msg.submit(lambda: "", None, msg)   # clears input
    send.click(chat_fn, [msg, chatbot], [chatbot, chatbot])
    send.click(lambda: "", None, msg)

    # Clear button resets everything
    clear.click(clear_fn, None, chatbot)

demo.launch(share=True)





/tmp/ipython-input-2926504110.py:45: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-2926504110.py:45: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://338e761ea14dd6c45f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
